In [47]:
import pandas as pd
import plotly

#### Generates simple sankey plot nothing fancy

Df with columns: comb, count, nr_inds

In [91]:
df = pd.read_csv('data/example_data.csv')
# For real data uncomment if available
#df = pd.read_excel('data/df_sank_red.xlsx')
df.columns = df.columns.str.lower()
df.head()

,comb,count,nr_inds
0,UTI_UTI,167301,2
1,MISSING_MISSING,138878,2
2,RTI_RTI,41296,2
3,UTI_MISSING,37213,2
4,MISSING_UTI,31285,2


In [75]:
df = pd.read_csv('data/example_data.csv')


In [92]:
df = df.assign(lvl1 = 'stop_1', lvl2 = 'stop_2', lvl3 = 'stop_3', lvl4 = 'stop_4', lvl5 = 'stop_5')
df.head()

,comb,count,nr_inds,lvl1,lvl2,lvl3,lvl4,lvl5
0,UTI_UTI,167301,2,stop_1,stop_2,stop_3,stop_4,stop_5
1,MISSING_MISSING,138878,2,stop_1,stop_2,stop_3,stop_4,stop_5
2,RTI_RTI,41296,2,stop_1,stop_2,stop_3,stop_4,stop_5
3,UTI_MISSING,37213,2,stop_1,stop_2,stop_3,stop_4,stop_5
4,MISSING_UTI,31285,2,stop_1,stop_2,stop_3,stop_4,stop_5


In [93]:
for tup in df.itertuples():
    i = tup.Index
    string_to_split = tup.comb
    list_of_v = string_to_split.split('_')
    for j in range(len(list_of_v)):
        if j > 4:
            break
        df.loc[i, 'lvl'+str(j+1)] = str(list_of_v[j])+ '_' + str(j+1)
            
        
        

In [81]:
df.head()

,comb,count,nr_inds,lvl1,lvl2,lvl3,lvl4,lvl5
0,v1_v1,10,2,v1_1,v1_2,nothing,nothing,nothing
1,v2_v1_v1_v1_v1_v1_v1_v1_v1_v1_v1,1,11,v2_1,v1_2,v1_3,v1_4,v1_5
2,v1_v2_v3,50,3,v1_1,v2_2,v3_3,nothing,nothing
3,v3_v3,3,2,v3_1,v3_2,nothing,nothing,nothing
4,v4_v4_v4_v4_v4,5,5,v4_1,v4_2,v4_3,v4_4,v4_5


In [120]:
def generate_sankey(df, cat_columns=[], value_cols='', title='Sankey Diagram', colors = ['#9b5de5','#f15bb5','#fee440','#00bbf9','#00f5d4']):
    label_list = []
    color_list = []
    for cat_col in cat_columns:
        label_list_temp = list(set(df[cat_col].values))
        color_list.append(len(label_list_temp))
        label_list = label_list + label_list_temp
        
    label_list = list(dict.fromkeys(label_list))

    color_list_2 = []
    for i, color_n in enumerate(color_list):
        color_list_2 = color_list_2 + [colors[i]]*color_n
    
    for i in range(len(cat_columns)-1):
        if i==0:
            source_df = df[[cat_columns[i],cat_columns[i+1], value_cols]]
            source_df.columns = ['source','target','count']
        else:
            temp_df = df[[cat_columns[i], cat_columns[i+1], value_cols]]
            temp_df.columns = ['source','target','count']
            source_df = pd.concat([source_df,temp_df])
        source_df = source_df.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    source_df['sourceID'] = source_df['source'].apply(lambda x: label_list.index(x))
    source_df['targetID'] = source_df['target'].apply(lambda x: label_list.index(x))
    
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = label_list,
          color = color_list_2
        ),
        link = dict(
          source = source_df['sourceID'],
          target = source_df['targetID'],
          value = source_df['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [121]:
fig = generate_sankey(df, cat_columns=['lvl1','lvl2', 'lvl3', 'lvl4', 'lvl5'],value_cols='count',title='Sankey Diagram')

['#9b5de5', '#9b5de5', '#9b5de5', '#9b5de5', '#9b5de5', '#f15bb5', '#f15bb5', '#f15bb5', '#f15bb5', '#f15bb5', '#fee440', '#fee440', '#fee440', '#fee440', '#fee440', '#fee440', '#00bbf9', '#00bbf9', '#00bbf9', '#00bbf9', '#00bbf9', '#00bbf9', '#00f5d4', '#00f5d4', '#00f5d4', '#00f5d4', '#00f5d4', '#00f5d4']


In [122]:
plotly.offline.plot(fig, validate=False)

'temp-plot.html'